In [ ]:
!python finetuning.py

In [1]:
!python finetuning2.py --dataset=MIMIC-CXR \
  --base_model_path=./base-models/simclr/r152_2x_sk1/hub/ \
  --epochs=10 --batch_size=64 --learning_rate=1.0

2024-01-30 15:23:24.659168: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
GPU: []
Dataset: MIMIC-CXR
Model: ./base-models/simclr/r152_2x_sk1/hub/
Epochs: 11
Batch size: 64
2024-01-30 15:23:41.385229: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 54 of 128
^C


In [1]:
import tensorflow as tf
print('list:', tf.config.list_physical_devices('GPU'))
print('available:', tf.test.is_gpu_available())

# with tf.device('/gpu:0'):
#     a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
#     b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
#     c = tf.matmul(a, b)
#     tf.config.list_physical_devices('GPU')

2024-01-30 21:23:37.724870: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


list: []
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
available: False


In [ ]:
# SETUP
# Copyright 2023 The medical_research_foundations Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Original code from medical_research_foundations repository in:
#   /colab/REMEDIS_finetuning_example.ipynb
# Modified to add support for MIMIC-CXR-JPG dataset

import re
import os
import numpy as np
from time import time

# import tensorflow.compat.v2 as tf
# tf.compat.v1.enable_v2_behavior()
# import tensorflow.compat.v1 as tf
# tf.disable_eager_execution()
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras import layers
import matplotlib
import matplotlib.pyplot as plt

# LARS optimizer from lars_optimizer.py in SimCLR repository
from lars_optimizer import LARSOptimizer
# macro_soft_f1 optimizer from multi-label-soft-f1 repository
from objective_func import macro_soft_f1, macro_f1
# Preprocessing functions from data_util.py in SimCLR repository
from utils.augmentation import preprocess_image
# utilities to plot, time, and score
from utils.analysis import *

from loader.mimic_cxr_jpg_loader import MIMIC_CXR_JPG_Loader


#------------------- PARAMS -------------------#
DATASET = "MIMIC-CXR" #@param ["Chexpert", "Camelyon", "MIMIC-CXR", "Noise"]
BASE_MODEL = "SimCLR" #@param ["SimCLR", "DINO", "REMEDIS"]
BATCH_SIZE = 5 #64
IMAGE_SIZE = (448,448)
CHANNELS = 3
NUM_SAMPLES = 64
LEARNING_RATE = 0.1
EPOCHS = 2 #10
MOMENTUM = 0.9
WEIGHT_DECAY = 0.


#------------------- VARS -------------------#
project_folder = os.getcwd()
# model_required_shapes = {
#   SimCLR: {image_size: (448,448), },
#   DINO: {},
#   REMEDIS: {}
#   }

#------------------- LOAD DATA -------------------#

# Chexpert: TFDS.has Supervised - produces binary labels the way we were using them
#           Chexpert data loader fails unless you have it downloaded - download & put in this directory
#           Have self-selectors
# Noise:    fake tfds for testing
def _preprocess_train(x, y, info=None):
  x = preprocess_image(
      x, *IMAGE_SIZE,
      is_training=True, color_distort=False, crop='Center')
  return x, y

def _preprocess_val(x, y, info=None):
  x = preprocess_image(
      x, *IMAGE_SIZE,
      is_training=False, color_distort=False, crop='Center')
  return x, y

if DATASET == 'Noise':
  def generate_fake_tfds_dataset(width, height, channels, num_classes, N=1000, data_type=tf.float32):
    train_examples = np.random.normal(size=[N, width, height, channels])
    classes = np.arange(0, num_classes)
    random_labels = np.random.choice(a=classes, size=N)
    one_hot_encoded = np.zeros((N, num_classes))
    one_hot_encoded[np.arange(N), random_labels] = 1
    train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, one_hot_encoded))
    return train_dataset
  
  num_classes = 14
  train_tfds = generate_fake_tfds_dataset(*IMAGE_SIZE, CHANNELS, num_classes, NUM_SAMPLES)
  
elif DATASET == 'Camelyon':
  # NOTE: This is too large to run with the public runtime. Run locally.
  # To see more information about the patch_camelyon dataset, see
  # (https://github.com/basveeling/pcam).
  train_tfds, tfds_info = tfds.load('patch_camelyon',
                                      split='train[:1%]',
                                      with_info = True,
                                      as_supervised = True)
  num_images = tfds_info.splits['train'].num_examples
  num_classes = tfds_info.features['label'].num_classes
  train_tfds = train_tfds.map(lambda x, y: (x, tf.one_hot(y, num_classes)))

elif DATASET == 'Chexpert':
  # TODO: Load chexpert data here.
  num_classes = 14
  raise Exception("not implemented. Please download the chexpert data manually and add code to read here.")

elif DATASET == 'MIMIC-CXR':
  customLoader = MIMIC_CXR_JPG_Loader({'train': NUM_SAMPLES, 'validate': NUM_SAMPLES, 'test': 0}, project_folder)
  train_tfds, val_tfds, test_tfds = customLoader.load()
  num_classes = customLoader.metadata['num_classes']

else:
  raise Exception('The Data Type specified does not have data loading defined.')

train_tfds = train_tfds.shuffle(buffer_size=2*BATCH_SIZE)
batched_train_tfds = train_tfds.map(_preprocess_train).batch(BATCH_SIZE)
# val_tfds = val_tfds.shuffle(buffer_size=2*BATCH_SIZE)
batched_val_tfds = val_tfds.map(_preprocess_val).batch(BATCH_SIZE)

# TODO: in case improves performance
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# batched_train_tfds = batched_train_tfds.prefetch(buffer_size=AUTOTUNE)

# next_batch = tf.data.make_one_shot_iterator(batched_train_tfds).get_next()

for f, l in batched_train_tfds.take(1):
  print("Shape of features array:", f.numpy().shape)
  print("Shape of labels array:", l.numpy().shape)

#------------------- LOAD MODLES -------------------#
# Load module and construct the computation graph
# Load the base network and set it to non-trainable (for speedup fine-tuning)

if BASE_MODEL == 'REMEDIS':
  hub_path = os.path.join(project_folder, './base-models/remedis/cxr-152x2-remedis-m/')
elif BASE_MODEL == 'SimCLR':
  hub_path = os.path.join(project_folder, './base-models/simclr/r152_2x_sk1/hub/')
elif BASE_MODEL == 'DINO':
  raise Exception('TODO: Not implemented yet.')
  hub_path = os.path.join(project_folder, './base-models/dino/???')

try:
  feature_extractor_layer = hub.KerasLayer(hub_path, input_shape=(*IMAGE_SIZE, CHANNELS), trainable=False)
except:
  print(f"""The model {hub_path} did not load. Please verify the model path. It is also worth considering that the model might still be in the process of being uploaded to the designated location. If you have recently uploaded it to a notebook, there could be delays associated with the upload.""")
  raise

#------------------- SETUP TRAINING HEAD -------------------#
model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(1024, activation='relu', name='hidden_layer'), # TODO: check with Leon, why 1024
    layers.Dense(num_classes, activation='sigmoid', name='multi-label_classifier')
])

# TEMP for debugging
print(model.summary())
for batch in batched_val_tfds:
    print('probability of an image for classes:', model.predict(batch[0])[:1])
    break

optimizer = tf.keras.optimizers.Adam(
  learning_rate=LEARNING_RATE,
  weight_decay=WEIGHT_DECAY)
optimizer.exclude_from_weight_decay(
    var_names=['batch_normalization', 'bias', 'head_supervised'])

model.compile(
  optimizer=optimizer,
  loss=macro_soft_f1,
  metrics=[macro_f1])

In [ ]:
# TRAIN
start = time()
history = model.fit(batched_train_tfds,
                    epochs=EPOCHS,
                    validation_data=batched_val_tfds)
print('\nTraining took {}'.format(print_time(time()-start)))

In [ ]:
# ANALYSE

from utils.analysis import *

losses, val_losses, macro_f1s, val_macro_f1s = learning_curves(history)
# model_bce_losses, model_bce_val_losses, model_bce_macro_f1s, model_bce_val_macro_f1s = learning_curves(history_bce)

for batch in batched_val_tfds:
  print('probability of an image for classes (after finetuning):', model.predict(batch[0])[:1])
  break

print("Macro soft-F1 loss: %.2f" %val_losses[-1])
print("Macro F1-score: %.2f" %val_macro_f1s[-1])

for batch in batched_val_tfds:
  show_prediction(*batch, model)
  break
